In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [2]:
pickles = ['A.pickle', 'B.pickle', 'C.pickle', 'D.pickle', 'E.pickle', 'F.pickle', 'G.pickle', 'H.pickle']

In [3]:
X = []
y = []

def append_data(pickle_file):
    global X
    global y
    df = pd.read_pickle(pickle_file)
    for ind, row in df[['segments_pitches', 'segments_timbre', 'segments_loudness_start', 'song_hotttnesss']].iterrows():
        if row['segments_pitches'].shape[0] >= 120:
            beg_ind = (row['segments_pitches'].shape[0]-120)//2
            pitch = row['segments_pitches'][beg_ind:beg_ind+120]
            timbre = row['segments_timbre'][beg_ind:beg_ind+120]
            loudness = row['segments_loudness_start'][beg_ind:beg_ind+120].reshape(-1, 1)
            y.append(row['song_hotttnesss'])
            X.append( np.concatenate((pitch, timbre, loudness), axis=1) )
        if ind % 10000 == 0: print(pickle_file, 'reached', ind)

In [4]:
X = []
y = []

for p in pickles:
    append_data(p)

A.pickle reached 0
A.pickle reached 30000
B.pickle reached 0
B.pickle reached 20000
C.pickle reached 10000
C.pickle reached 30000
D.pickle reached 0
D.pickle reached 10000
D.pickle reached 20000
D.pickle reached 30000
E.pickle reached 20000
F.pickle reached 0
F.pickle reached 10000
G.pickle reached 20000
G.pickle reached 30000
H.pickle reached 10000
H.pickle reached 30000


In [5]:
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

(176825, 120, 25) (176825,)


In [6]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=5000, random_state=42)

print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

(161825, 120, 25) (161825,) (5000, 120, 25) (5000,) (10000, 120, 25) (10000,)


In [7]:
mean = [X_train[:, :, :12].mean(), X_train[:, :, 12:24].mean(), X_train[:, :, 24].mean()]
std = [X_train[:, :, :12].std(), X_train[:, :, 12:24].std(), X_train[:, :, 24].std()]

X_train[:, :, :12] = ( X_train[:, :, :12] - mean[0] ) / std[0]
X_train[:, :, 12:24] = ( X_train[:, :, 12:24] - mean[1] ) / std[1]
X_train[:, :, 24] = ( X_train[:, :, 24] - mean[2] ) / std[2]

X_valid[:, :, :12] = ( X_valid[:, :, :12] - mean[0] ) / std[0]
X_valid[:, :, 12:24] = ( X_valid[:, :, 12:24] - mean[1] ) / std[1]
X_valid[:, :, 24] = ( X_valid[:, :, 24] - mean[2] ) / std[2]

X_test[:, :, :12] = ( X_test[:, :, :12] - mean[0] ) / std[0]
X_test[:, :, 12:24] = ( X_test[:, :, 12:24] - mean[1] ) / std[1]
X_test[:, :, 24] = ( X_test[:, :, 24] - mean[2] ) / std[2]

In [8]:
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_valid', X_valid)
np.save('y_valid', y_valid)
np.save('X_test', X_test)
np.save('y_test', y_test)